In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
from sklearn.metrics import roc_auc_score

%pylab inline

date_columns = [204, 75, 73, 217, 158, 159, 156, 157, 176, 177, 178, 179, 166, 167, 168, 169]
def parse_date(x):
    return pd.to_datetime(x, format="%d%b%y:%H:%M:%S") 

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
data = pd.read_csv('train.csv')

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
print(data['target'].sum())
print(len(data.index))
data.info()

33773
145231
<class 'pandas.core.frame.DataFrame'>
Int64Index: 145231 entries, 0 to 145230
Columns: 1934 entries, ID to target
dtypes: float64(477), int64(1406), object(51)
memory usage: 2.1+ GB


In [4]:
train = data \
    .select_dtypes(include=['int64', 'float64']) \
    .drop(['ID', 'target'], axis=1) \
    .fillna(-1)

In [19]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

a_train, a_test, b_train, b_test = train_test_split(train, data['target'], test_size=0.2)

clf = RandomForestClassifier(n_estimators=20)

clf.fit(a_train, b_train)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
clf.predict_proba(a_test)[:, 1]

array([ 0.3,  0.1,  0.3, ...,  0.2,  0.2,  0.2])

In [14]:

test_pred = clf.predict(a_test)
print("    Score: %1.3f" % clf.score(a_test, b_test))
print("Precision: %1.3f" % precision_score(b_test, test_pred))
print("   Recall: %1.3f" % recall_score(b_test, test_pred))
print("       F1: %1.3f" % f1_score(b_test, test_pred))
print("      ROC: %1.3f" % roc_auc_score(b_test, test_pred))

    Score: 0.774
Precision: 0.561
   Recall: 0.196
       F1: 0.291
      ROC: 0.574


In [11]:

predictions = clf.predict(a_test)



0.57223710235876679

In [ ]:
test_data = pd.read_csv('test.csv',                    
                   parse_dates=date_columns,
                   #nrows=50000,
                   date_parser = parse_date)

In [ ]:
test_targets =  pd.Series(clf.predict(test_data[train.columns].fillna(-1)))

In [ ]:
submission = pd.concat([test_data['ID'], test_targets], axis=1)
submission.columns = ['ID', 'target']
submission.to_csv('submission.csv', index=False)